## 설계 순서
->

    1. pre-trained 모델 이용해서 train데이터에 대해 학습.
    
    2. 왜곡 푸는 함수 파트(반드시 평면이어야 되는건 아님)
   
    3. 다시 왜곡 사진으로 바꾸는 함수
    
    4. test에 대해 예측


In [ ]:
!pip install transformers
!pip3 install natten -f https://shi-labs.com/natten/wheels/cu118/torch2.0.0/index.html

In [ ]:
from transformers import OneFormerProcessor, OneFormerForUniversalSegmentation
from PIL import Image
import pandas as pd
from tqdm import tqdm
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
processor = OneFormerProcessor.from_pretrained("shi-labs/oneformer_cityscapes_dinat_large")
model = OneFormerForUniversalSegmentation.from_pretrained("shi-labs/oneformer_cityscapes_dinat_large").to('cuda')

## 드라이브 연결


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
cd /content/drive/MyDrive/open

In [ ]:
# RLE 인코딩 함수
# RLE(Run-Length Encoding)은 연속적으로 반복되는 값을 압축하는 방법 중 하나입니다.

def rle_encode(mask):
    # 입력된 이진 마스크를 1차원 배열로 변환합니다.
    pixels = mask.flatten()

    # 배열의 시작과 끝에 0을 추가하여 RLE 알고리즘을 적용하기 위한 기본 준비를 합니다.
    pixels = np.concatenate([[0], pixels, [0]])

    # 연속하지 않는 값의 인덱스를 찾아내어 압축합니다.
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1

    # 압축된 런 길이를 계산합니다.
    runs[1::2] -= runs[::2]

    # 압축된 런을 문자열로 변환하여 반환합니다.
    return ' '.join(str(x) for x in runs)

## customdataset


In [ ]:
# 사용자 정의 데이터셋 클래스
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        # 주어진 CSV 파일을 읽어 데이터를 로드합니다.
        self.data = pd.read_csv(csv_file)

        # 데이터 변환(transform) 및 추론 모드 여부(infer)를 설정합니다.
        self.transform = transform
        self.infer = infer

    def __len__(self):
        # 데이터셋의 길이를 반환합니다.
        return len(self.data)

    def __getitem__(self, idx):
        # 주어진 인덱스(idx)를 사용하여 이미지 및 마스크 파일 경로를 가져옵니다.
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            # 추론 모드(infer)인 경우, 이미지 변환(transform)을 적용하고 이미지만 반환합니다.
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_path = self.data.iloc[idx, 2]
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # 마스크 이미지에서 픽셀 값이 255인 부분을 12로 간주하여 변경합니다.
        mask[mask == 255] = 12

        if self.transform:
            # 훈련 또는 검증 모드인 경우, 이미지와 마스크에 변환을 적용합니다.
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        # 이미지와 마스크를 반환합니다.
        return image, mask

In [ ]:
# 데이터 변환 파이프라인을 설정합니다. 이 파이프라인은 이미지에 대한 전처리 작업을 정의합니다.
#albumentations를 A로 두고 transform사용 -> albumentations는 torchvision보다 더 다양하고 빠른 모델임..
transform = A.Compose(

    [
        # 이미지를 224x224 크기로 조정합니다.
        A.Resize(224, 224),

        # 이미지를 정규화합니다. (평균과 표준편차를 사용하여 스케일링)
        A.Normalize(),

        # 이미지를 PyTorch 텐서로 변환합니다.
        ToTensorV2()
    ]
)



In [ ]:
# 대회에서 제공하는 train 데이터셋을 로드합니다.
train_dataset = CustomDataset(csv_file='train_source.csv', transform=transform)

# 데이터로더를 설정합니다.
dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)

# loss function과 optimizer 정의
criterion = torch.nn.CrossEntropyLoss()  # Cross Entropy 손실 함수를 정의합니다.
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam 옵티마이저를 정의하고 모델의 파라미터를 최적화합니다.

# training loop
for epoch in range(20):  # 20 에폭 동안 학습을 반복합니다.
    model.train()  # 모델을 학습 모드로 설정합니다.
    epoch_loss = 0  # 현재 에폭의 손실을 저장할 변수를 초기화합니다.

    for images, masks in tqdm(dataloader):
        images = images.float().to(device)  # 이미지 데이터를 GPU로 전송합니다.
        masks = masks.long().to(device)  # 마스크 데이터를 GPU로 전송합니다.

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks.squeeze(1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(dataloader)}')

In [ ]:
cityscapes_to_12 = {
    0: [6,7],
    1: [8],
    2: [11,12,13,15],
    3: [17,18,14],
    4: [1],
    5: [19],
    6: [20],
    7: [21],
    8: [23],
    9: [24,25],
    10: [],
    11: [26,27,28,29,30,32,33]
}

## 추론

In [ ]:
df = pd.read_csv('test.csv')
df.head()

In [ ]:
submit = pd.read_csv('./sample_submission.csv')


test를 예측하는 부분(이 전에 test에 대한 처리하는 코드가 와야함)

In [ ]:
result = []
for i in tqdm(range(len(df))):
    image = Image.open(df['img_path'][i])
    # image resize
    image = image.resize((960, 540))

    semantic_inputs = processor(images=image, task_inputs=["semantic"], return_tensors="pt")

    for key in semantic_inputs.keys():
        semantic_inputs[key] = semantic_inputs[key].to('cuda')

    semantic_outputs = model(**semantic_inputs)

    # pass through image_processor for postprocessing
    predicted_semantic_map = processor.post_process_semantic_segmentation(semantic_outputs, target_sizes=[image.size[::-1]])[0]

    del semantic_inputs, semantic_outputs
    predicted_semantic_map_np = np.array(predicted_semantic_map.cpu().numpy())

    # convert to rle
    for key, value in cityscapes_to_12.items():
        key_mask = np.isin(predicted_semantic_map_np, value)
        if np.sum(key_mask) > 0:
            mask_rle = rle_encode(key_mask)
            result.append(mask_rle)
        else:
            result.append(-1)

결과 중에 임의의 한장의 마스크값을 다 합쳐서 이미지를 출력하는 부분(될지 안될지는 gpu터져서 모름)

In [ ]:
import random

# 랜덤한 클래스 색상 지정
class_colors = {}
for class_id in range(12):
    # 랜덤한 RGB 값 생성
    color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    class_colors[class_id] = color

# 모든 클래스의 마스크를 합치기 위한 빈 이미지 생성
merged_mask = np.zeros_like(predicted_semantic_map_np)

# 모든 클래스에 대한 마스크를 합치고 랜덤한 색상 적용
for class_id, color in class_colors.items():
    class_mask = np.isin(predicted_semantic_map_np, [class_id]).astype(np.uint8) * 255
    merged_mask[class_mask > 0] = color

# 이미지와 합친 마스크를 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(image)
axes[0].set_title("Original Image")
axes[0].axis('off')
axes[1].imshow(merged_mask)
axes[1].set_title("Merged Class Masks")
axes[1].axis('off')
plt.show()

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['mask_rle'] = result
submit

In [ ]:
submit.to_csv('segformer_pretrain_submit.csv', index=False)